# Binary Tree (with optional Binary Search Tree behavior)

A binary tree is a hierarchical data structure where each node has at most two children: `left` and `right`.

This notebook provides:
- A generic `BinaryTree` struct with fundamental operations and traversals.
- An optional BST mode (`asBST=true`) where inserts/searches enforce the Binary Search Tree ordering.
- Simple, well-typed Go implementation, plus sample usage for both generic and BST modes.

Notes:
- Duplicates in BST mode go to the right subtree by convention here.
- You can pass a comparison function to define the ordering for complex objects.
- In generic mode, you can manually attach children or use level-order `Insert()` to fill the next available slot.


In [1]:
package main

import (
    "fmt"
)

// Node represents a binary tree node
type Node[T any] struct {
    Value T
    Left  *Node[T]
    Right *Node[T]
}

// BinaryTree represents a binary tree with optional BST behavior
type BinaryTree[T any] struct {
    Root *Node[T]
    AsBST bool
    Compare func(T, T) int // -1 if a < b, 0 if a == b, 1 if a > b
}

// NewBinaryTree creates a new binary tree
func NewBinaryTree[T any](asBST bool, compare func(T, T) int) *BinaryTree[T] {
    return &BinaryTree[T]{
        Root: nil,
        AsBST: asBST,
        Compare: compare,
    }
}

// Insert adds a value to the tree
func (bt *BinaryTree[T]) Insert(value T) {
    if bt.Root == nil {
        bt.Root = &Node[T]{Value: value}
        return
    }
    
    if bt.AsBST {
        bt.insertBST(bt.Root, value)
    } else {
        bt.insertLevelOrder(value)
    }
}

// insertLevelOrder inserts in level-order (breadth-first)
func (bt *BinaryTree[T]) insertLevelOrder(value T) {
    queue := []*Node[T]{bt.Root}
    
    for len(queue) > 0 {
        curr := queue[0]
        queue = queue[1:]
        
        if curr.Left == nil {
            curr.Left = &Node[T]{Value: value}
            return
        }
        queue = append(queue, curr.Left)
        
        if curr.Right == nil {
            curr.Right = &Node[T]{Value: value}
            return
        }
        queue = append(queue, curr.Right)
    }
}

// insertBST inserts maintaining BST ordering
func (bt *BinaryTree[T]) insertBST(node *Node[T], value T) {
    cmp := bt.Compare(value, node.Value)
    if cmp < 0 {
        if node.Left == nil {
            node.Left = &Node[T]{Value: value}
        } else {
            bt.insertBST(node.Left, value)
        }
    } else { // duplicates go right
        if node.Right == nil {
            node.Right = &Node[T]{Value: value}
        } else {
            bt.insertBST(node.Right, value)
        }
    }
}

// Search finds a node with the given value
func (bt *BinaryTree[T]) Search(value T) *Node[T] {
    if bt.Root == nil {
        return nil
    }
    
    if bt.AsBST {
        return bt.searchBST(bt.Root, value)
    }
    return bt.searchLinear(value)
}

// searchBST searches using BST ordering
func (bt *BinaryTree[T]) searchBST(node *Node[T], value T) *Node[T] {
    if node == nil {
        return nil
    }
    
    cmp := bt.Compare(value, node.Value)
    if cmp == 0 {
        return node
    } else if cmp < 0 {
        return bt.searchBST(node.Left, value)
    }
    return bt.searchBST(node.Right, value)
}

// searchLinear searches linearly through all nodes
func (bt *BinaryTree[T]) searchLinear(value T) *Node[T] {
    nodes := bt.LevelOrderNodes()
    for _, node := range nodes {
        if bt.Compare(value, node.Value) == 0 {
            return node
        }
    }
    return nil
}

// Contains checks if a value exists in the tree
func (bt *BinaryTree[T]) Contains(value T) bool {
    return bt.Search(value) != nil
}

// Preorder traversal: Node -> Left -> Right
func (bt *BinaryTree[T]) Preorder() []T {
    var result []T
    bt.preorderHelper(bt.Root, &result)
    return result
}

func (bt *BinaryTree[T]) preorderHelper(node *Node[T], result *[]T) {
    if node == nil {
        return
    }
    *result = append(*result, node.Value)
    bt.preorderHelper(node.Left, result)
    bt.preorderHelper(node.Right, result)
}

// Inorder traversal: Left -> Node -> Right
func (bt *BinaryTree[T]) Inorder() []T {
    var result []T
    bt.inorderHelper(bt.Root, &result)
    return result
}

func (bt *BinaryTree[T]) inorderHelper(node *Node[T], result *[]T) {
    if node == nil {
        return
    }
    bt.inorderHelper(node.Left, result)
    *result = append(*result, node.Value)
    bt.inorderHelper(node.Right, result)
}

// Postorder traversal: Left -> Right -> Node
func (bt *BinaryTree[T]) Postorder() []T {
    var result []T
    bt.postorderHelper(bt.Root, &result)
    return result
}

func (bt *BinaryTree[T]) postorderHelper(node *Node[T], result *[]T) {
    if node == nil {
        return
    }
    bt.postorderHelper(node.Left, result)
    bt.postorderHelper(node.Right, result)
    *result = append(*result, node.Value)
}

// LevelOrder returns values in level-order
func (bt *BinaryTree[T]) LevelOrder() []T {
    nodes := bt.LevelOrderNodes()
    result := make([]T, len(nodes))
    for i, node := range nodes {
        result[i] = node.Value
    }
    return result
}

// LevelOrderNodes returns nodes in level-order
func (bt *BinaryTree[T]) LevelOrderNodes() []*Node[T] {
    if bt.Root == nil {
        return nil
    }
    
    var result []*Node[T]
    queue := []*Node[T]{bt.Root}
    
    for len(queue) > 0 {
        curr := queue[0]
        queue = queue[1:]
        result = append(result, curr)
        
        if curr.Left != nil {
            queue = append(queue, curr.Left)
        }
        if curr.Right != nil {
            queue = append(queue, curr.Right)
        }
    }
    return result
}

// Height returns the height of the tree
func (bt *BinaryTree[T]) Height() int {
    return bt.heightHelper(bt.Root)
}

func (bt *BinaryTree[T]) heightHelper(node *Node[T]) int {
    if node == nil {
        return -1 // height of empty tree
    }
    leftHeight := bt.heightHelper(node.Left)
    rightHeight := bt.heightHelper(node.Right)
    if leftHeight > rightHeight {
        return 1 + leftHeight
    }
    return 1 + rightHeight
}

// IsEmpty checks if the tree is empty
func (bt *BinaryTree[T]) IsEmpty() bool {
    return bt.Root == nil
}

// Clear removes all nodes from the tree
func (bt *BinaryTree[T]) Clear() {
    bt.Root = nil
}

// Helper function for integer comparison
func intCompare(a, b int) int {
    if a < b {
        return -1
    } else if a > b {
        return 1
    }
    return 0
}

## Examples

Below are quick examples showing generic mode (level-order inserts) and BST mode (ordered inserts and search).


In [2]:
%%
// Generic mode: level-order insertion
bt := NewBinaryTree(false, intCompare)
values := []int{1, 2, 3, 4, 5, 6}
for _, v := range values {
    bt.Insert(v)
}

/*
tree
         1
        / \
       2   3
     / \  /
    4  5 6
*/

fmt.Println("Generic tree level-order:", bt.LevelOrder())
fmt.Println("Preorder:", bt.Preorder())
fmt.Println("Inorder:", bt.Inorder())
fmt.Println("Postorder:", bt.Postorder())
fmt.Println("Height:", bt.Height())
fmt.Println("Contains 5?", bt.Contains(5))
fmt.Println("Contains 42?", bt.Contains(42))

Generic tree level-order: [1 2 3 4 5 6]
Preorder: [1 2 4 5 3 6]
Inorder: [4 2 5 1 6 3]
Postorder: [4 5 2 6 3 1]
Height: 2
Contains 5? true
Contains 42? false


In [3]:
%%
// BST mode: ordered insert/search
bst := NewBinaryTree(true, intCompare)
bstValues := []int{8, 3, 10, 1, 6, 14, 4, 7, 13}
for _, v := range bstValues {
    bst.Insert(v)
}

fmt.Println("BST inorder (sorted):", bst.Inorder())
fmt.Println("BST level-order:", bst.LevelOrder())
fmt.Println("Search 7:", bst.Search(7) != nil)
fmt.Println("Search 2:", bst.Search(2) != nil)
fmt.Println("Height:", bst.Height())

BST inorder (sorted): [1 3 4 6 7 8 10 13 14]
BST level-order: [8 3 10 1 6 14 4 7 13]
Search 7: true
Search 2: false
Height: 3


In [6]:
// Using custom comparison for structs
type Person struct {
    Name string
    Age  int
}

func personAgeCompare(a, b Person) int {
    return intCompare(a.Age, b.Age)
}

%%
peopleBST := NewBinaryTree(true, personAgeCompare)
people := []Person{
    {"Ann", 30},
    {"Ben", 25},
    {"Cara", 40},
}

for _, p := range people {
    peopleBST.Insert(p)
}

fmt.Print("People ages inorder (sorted by age): ")
for _, p := range peopleBST.Inorder() {
    fmt.Printf("%d ", p.Age)
}
fmt.Println()

searchPerson := Person{"X", 25}
found := peopleBST.Search(searchPerson)
fmt.Println("Search by age=25:", found != nil)
if found != nil {
    fmt.Printf("Found: %+v\n", found.Value)
}

People ages inorder (sorted by age): 25 30 40 
Search by age=25: true
Found: {Name:Ben Age:25}
